In [9]:
import transformers
from transformers.pipelines import PIPELINE_REGISTRY
from pipeline import NER_Pipeline
from selenium import webdriver
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from faker import Faker
import time

from server import get_acronym_description, request_results, get_acronyms_and_long_forms

In [2]:
# Register custom pipeline
PIPELINE_REGISTRY.register_pipeline(
    "NER_NLP_tagger",
    pipeline_class=NER_Pipeline,
    pt_model=transformers.AutoModelForTokenClassification
)

# Load NER tagger pipeline
ner_tagger = transformers.pipeline(
    "NER_NLP_tagger", model="SurtMcGert/NLP-group-CW-roberta-ner-tagging")

In [3]:
def compare(expected, predicted):
    if (expected == predicted):
        print("============ Test passed ============")
    else:
        print(f"============ Test failed ============\nPredicted:\n{predicted}")

Test to see if the `ner_tagger` model is called correctly by the server

In [ ]:
input = "This is a test"

expected = ner_tagger(input)

predicted = request_results(input)

compare(expected, predicted)

Test to see if the `get_acronym_description()` function works as expected

In [5]:
input = "Federal Bureau of Investigation (FBI)"

expected = "The Federal Bureau of Investigation (FBI) is the domestic intelligence and security service of the United States and its principal federal law enforcement agency"

predicted = get_acronym_description(input)

if (predicted == ""):
    print(True)
else:
    compare(expected, predicted)

============ Test passed ============


Test to see if the `get_acronym_and_long_forms()` function works as expected

In [6]:
# Test to see if get_acronyms_and_long_forms is functional
input = [("This", "B-O"), ("is", "B-O"), ("a", "B-O"), ("test", "B-O"), ("Federal", "B-LF"), ("Bureau", "I-LF"), ("of", "I-LF"), ("Investigation", "I-LF"), ("FBI", "B-AC")]

expected = {
    "Acronyms": ["FBI"],
    "Long Forms": ["Federal Bureau of Investigation"]
}

predicted = get_acronyms_and_long_forms(input)

compare(expected, predicted)

============ Test passed ============


Test the front-end functionality. This test will create a list of randomly-generated text, and feed it through an actual browser using the Selenium library.

In [ ]:
# Test server's responsiveness
test_input = []

# Create a list of randomly-generated text
fake = Faker()
for i in range(5):
    test_input.append(fake.text(max_nb_chars=2000))

# Create a list of the page elements we want to find on the results page
xpaths = [
    "/html/body/div[1]/div[1]/div[1]/div/div/div/section/div[1]/div/div/div/div[6]/div[2]/div/div/div/div/div/div/table",
    "/html/body/div[1]/div[1]/div[1]/div/div/div/section/div[1]/div/div/div/div[4]/div/div/p/span",
    "/html/body/div[1]/div[1]/div[1]/div/div/div/section/div[1]/div/div/div/div[6]/div[2]/div/div/div/div/div/div/table"
]

# Create web browser
browser = webdriver.Firefox()

# Go to website
browser.get("http://localhost:8501")

# Wait for website to load
time.sleep(5)

test_pass = True
for i, input_string in enumerate(test_input):
    # Find the input text box
    element = WebDriverWait(browser, 20).until(EC.element_to_be_clickable((By.XPATH, "//*[@id=\"text_input_1\"]")))
    # Since this test runs repeatedly, clear the text box of the last test input
    element.clear()
    # Input the sample text
    element.send_keys(input_string)
    # Press the Enter key
    element.send_keys(Keys.RETURN)

    # Look to see if the elements are generated correctly
    for xpath in xpaths:
        try:
            element = WebDriverWait(browser, 20).until(EC.presence_of_element_located((By.XPATH, xpath)))
        except:
            # Fail the test if it cannot find one
            test_pass = False

    # Pause just so we can see the results page
    time.sleep(2)

# Close the browser
browser.quit()

if (test_pass == True):
    print("Test passed")
else:
    print("Test failed")